# Trie problem sq

In [78]:
class Node:
    def __init__(self):
        self.links = [None] * 26  # Array to store child nodes for each letter (26 lowercase letters)
        self.suggestions = []  # List to store lexicographically smallest suggestions

    def containsKey(self, ch):
        return self.links[ord(ch) - ord('a')] is not None
    def put(self, ch, node):
        self.links[ord(ch) - ord('a')] = node
    def get(self, ch):
        return self.links[ord(ch) - ord('a')]   


class Trie:
    def __init__(self):
        self.root = Node()

    # Insert a word into the Trie, maintaining up to 3 lexicographically smallest suggestions
    def insert(self, word):
        node = self.root
        for ch in word:
            if not node.containsKey(ch):
                node.put(ch, Node())
            node = node.get(ch)
            if len(node.suggestions) < 3:
                node.suggestions.append(word)

      

    # Get suggestions for each prefix of the searchWord
    def getSuggestions(self, searchWord):
        node = self.root
        results = []
        for ch in searchWord:
            if node.containsKey(ch):
                node = node.get(ch)
                results.append(node.suggestions)
            else:
                # If the prefix isn't found, append empty lists for remaining characters
                while len(results) < len(searchWord):
                    results.append([])
                break
        return results

# Function to get product suggestions based on a search word
def suggestedProducts(products, searchWord):
    # Sort products lexicographically
    products.sort()
    
    # Create a Trie and insert products into it
    trie = Trie()
    for product in products:
        trie.insert(product)
    
    # Retrieve suggestions for each prefix of the search word
    return trie.getSuggestions(searchWord)

# Example usage
products = ["mobile", "mouse", "moneypot", "monitor", "mousepad"]
searchWord = "mouse"
suggestedProducts(products, searchWord)



[['mobile', 'moneypot', 'monitor'],
 ['mobile', 'moneypot', 'monitor'],
 ['mouse', 'mousepad'],
 ['mouse', 'mousepad'],
 ['mouse', 'mousepad']]

# order book prob

In [78]:
class Order:
    def __init__(self, is_buy, qty, price):
        self.is_buy = is_buy  # True if it's a buy order, False if it's a sell order
        self.qty = qty        # The quantity of shares
        self.price = price    # The price per share

    def __str__(self):
        order_type = "buy" if self.is_buy else "sell"
        return f"{order_type} {self.qty}@$${self.price}"


class OrderBook:
    def __init__(self):
        self.orders = []  # List to store all orders (both buy and sell)

    def add(self, order):
        # Try to find the best matching trade
        while True:
            best_trade = self.find_trade(order)
            # If there's no trade, add the order to the order book
            if best_trade is None:
                
                break
            else:
                # Perform the trade: match quantities, remove the matched order if needed
                if order.qty == best_trade.qty:
                    self.orders.remove(best_trade)
                    self.orders.remove(order)
                    break
                elif order.qty < best_trade.qty:
                    best_trade.qty -= order.qty
                    self.orders.remove(order)
                    self.orders.sort(key=lambda x: (-x.price if x.is_buy else x.price, -x.qty ))
                    break
                else:
                    order.qty -= best_trade.qty  # Reduce the buy order's quantity
                    self.orders.remove(best_trade)
                    self.orders.sort(key=lambda x: (-x.price if x.is_buy else x.price, -x.qty ))
                    break             
                
    def find_trade(self, order):
        best_trade = None

        # Iterate through orders to find the best trade
        for existing_order in self.orders:
            if order.is_buy != existing_order.is_buy:
                # Buy order: find the lowest sell price that is acceptable
                if order.is_buy and order.price >= existing_order.price:
                    return existing_order

                # Sell order: find the highest buy price that is acceptable
                elif not order.is_buy and order.price <= existing_order.price:
                    return existing_order

        return best_trade  # Return the best trade if found, otherwise None

    def parse(self):
        # First collect all the orders
        collected_orders = []
        while True:
            line = input().strip()  # Read input from the user
            if line == "end":
                break  # Stop when "end" is encountered

            parts = line.split()
            order_type = parts[0] == "buy"  # Determine if it's a buy or sell order
            qty = int(parts[1])             # Read the quantity
            price = float(parts[2])         # Read the price

            order = Order(order_type, qty, price)  # Create the Order object
            self.orders.append(order)  # Collect the order

        self.orders.sort(key=lambda x: (-x.price if x.is_buy else x.price, -x.qty))
        for order in self.orders:
            self.add(order)
        self.orders.sort(key=lambda x: (-x.price if x.is_buy else x.price, -x.qty))
        
    def __str__(self):
        output = []
        for order in self.orders:
            output.append(str(order))
        return "\n".join(output)


# Example of how to use the class
if __name__ == "__main__":
    book = OrderBook()
    book.parse()
    print(book)  # Print the state of the order book after processing all orders


buy 10 13.0
buy 10 12.0
sell 15 12.0
sell 11 12.0
end
sell 5@$$12.0
sell 1@$$12.0


# unique usernames

In [1]:
import re

def generate_usernames(names):
    existing_usernames = set()  # To track already used usernames
    usernames = []
    
    for name in names:
        # Step 1: Extract given name and family name
        name_parts = name.replace("-", "").replace("'", "").split()  # Remove non-alphabetic characters
        given_name = name_parts[0].lower()  # First part is the given name
        family_name = name_parts[-1].lower()  # Last part is the family name
        
        # Step 2: Create default username
        base_family_part = family_name[:7]  # Take up to 7 characters of family name
        remaining_length = 8 - len(base_family_part)  # Remaining length to be filled by given name
        base_given_part = given_name[:remaining_length]  # Take as many chars as possible from given name
        
        default_username = base_family_part + base_given_part
        unique_username = default_username[:8]  # Ensure username doesn't exceed 8 characters

        # Step 3: Handle duplicates
        if unique_username in existing_usernames:
            # Rule 1: Try to adjust the balance between family and given name
            for i in range(1, len(given_name)):
                family_part = family_name[:7 - i]  # Reduce family name by i characters
                given_part = given_name[:8 - len(family_part)]  # Add more from given name
                unique_username = (family_part + given_part)[:8]  # Ensure it's 8 characters
                if unique_username not in existing_usernames:
                    break

            # Rule 2: If still not unique, replace the last character(s) with a digit
            if unique_username in existing_usernames:
                counter = 1
                while True:
                    replacement_username = unique_username[:7] + str(counter)  # Replace last char(s) with a digit
                    if replacement_username not in existing_usernames:
                        unique_username = replacement_username
                        break
                    counter += 1

        # Step 4: Add the unique username to the result list
        usernames.append(unique_username)
        existing_usernames.add(unique_username)  # Mark it as used

    return usernames

# Example usage:
names = [
    "Peter Parkers","Peter Parker","Peter Parker","Peter Parker","Peter Parker","Peter Parker","Peter Parker","Peter Parker","Peter Parker","Peter Parker","Peter Parker","Peter Parker",
    "Penelope Parker", "Penelope Parker", "Penelope Parker", "Penelope Parker", "Penelope Parker", "Penelope Parker", "Penelope Parker", 
    "Bruce Wayne", 
    "Clark Kent", 
    "Diana Prince","Diana Prince","Diana Prince","Diana Prince","Diana Prince","Diana Prince","Diana Prince","Diana Prince",
    "Peter O'N",  "Peter O'N"
]

print(generate_usernames(names))


['parkersp', 'parkerpe', 'parkepet', 'parkpete', 'parpeter', 'parpete1', 'parpete2', 'parpete3', 'parpete4', 'parpete5', 'parpete6', 'parpete7', 'parkepen', 'parkpene', 'parpenel', 'papenelo', 'ppenelop', 'penelope', 'penelop1', 'waynebru', 'kentclar', 'princedi', 'princdia', 'prindian', 'pridiana', 'pridian1', 'pridian2', 'pridian3', 'pridian4', 'onpeter', 'onpeter1']


# url hash

In [80]:
def hash_url(url, hash_string, k):
    # Step 1: Divide the URL into blocks of size k
    blocks = []
    for i in range(0, len(url), k):
        blocks.append(url[i:i+k])
    
    # Step 2: Calculate hash value for each block
    def calculate_hash(block):
        hash_value = 0
        for char in block:
            if 'a' <= char <= 'z':  # English lowercase letters
                hash_value += ord(char) - ord('a')
            elif 'A' <= char <= 'Z':  # English uppercase letters
                hash_value += ord(char) - ord('A')
            elif char == '/':
                hash_value += 26
            elif char == ':':
                hash_value += 27
            elif char == '.':
                hash_value += 28
            else:
                # Handle other characters based on requirements
                pass
        return hash_value
    print(blocks)
    
    ans=""
    for block in blocks:
        hash_value = calculate_hash(block)
        # Step 4: Replace the block with (hash_value % m)th character of hash_string
        mod_value = hash_value % len(hash_string)
        hashed_char = hash_string[mod_value]
        ans+= hashed_char

    return ans

# Example usage:
url = "https://xyz.comss"
hash_string = "abcdef"
k = 7

hashed_url = hash_url(url, hash_string, k)
print("Hashed URL:", hashed_url)


['https:/', '/xyz.co', 'mss']
Hashed URL: fea


# tick size

In [81]:
def process_raw_data(raw):
    # Step 1: Remove the second column
    processed_data = [[row[0], row[2]] for row in raw]

    final_data = []
    previous_value = None
    
    for row in processed_data:
        current_value = row[1]  # This corresponds to the former third column
        if current_value != previous_value:
            final_data.append([row[0], row[1]])  # Append both values as a sublist
            previous_value = current_value
    
    return final_data

# Sample input representing tick size raw data
raw = [
    [0, 0.1, 0.01],
    [0.1, 0.5, 0.02],
    [0.5, 1, 0.02],
    [1, 10, 0.02],
    [10, 50, 1],
    [50, -1, -1],  # Only the last row has -1
]

# Process the raw data
x=process_raw_data(raw)
x

[[0, 0.01], [0.1, 0.02], [10, 1], [50, -1]]

In [65]:
def func(arr, x):
    new=[]
    for num in arr:
        if num>= x[-1][0]:
            new.append('')
            
        for i,j in enumerate(x):
            if num< j[0]:
                new.append(num+ x[i-1][1] )
                break
  
    return new
func([50, 1], x)

['', 2]

# product sliding window

In [75]:
def func(arr, k):
    l=0; count=0; prod=1; n=len(arr)
    
    for i in range(n):
        prod*=arr[i]
        while prod>k:
            prod= prod//arr[l]
            l+=1
        count+=i-l+1
    return count
func([2,4,2,5,3,2,7,2], 10)       

12

In [2]:
def flip_switches(ranges):
    # Determine the maximum index of switches based on the input ranges
    max_index = max(end for _, end in ranges)
    
    # Initialize the difference array of size max_index + 2 (to handle range properly)
    switches = [0] * (max_index + 2)
    
    # Mark the flipping points
    for start, end in ranges:
        switches[start] += 1  # Start flipping from 'start'
        switches[end + 1] -= 1  # Stop flipping after 'end'

    # Compute the final states of the switches using a prefix sum approach
    current_state = 0
    on_switches = []
    
    for i in range(max_index + 1):
        current_state += switches[i]
        if current_state % 2 == 1:
            on_switches.append(i)

    return on_switches

# Example usage:
ranges = [[0, 3], [2, 10], [4, 5]]  # List of ranges (start, end)
result = flip_switches(ranges)
print(result)  # Output: indices of switches that are 'on'


[0, 1, 6, 7, 8, 9, 10]
